In [404]:
import pandas as pd # Data Analysis library for working with tabular data

# Load our AtlasIV data from the CSV file
df = pd.read_csv('AtlasIV.csv')
df.head()

,day,regen,battery,reflector,distance,data,radiation
0,1,N,89.4,5.0,1296,241,10.3
1,2,N,88.6,NaN,1291,254,10.2
2,3,Y,91.9,1.0,1334,169,10.2
3,4,N,89.9,NaN,1301,214,10.3
4,5,Y,90.5,4.0,1203,204,10.3


In [405]:
# Clean the regen column to be 0 for no recharge or 1 for recharged
df['regen'].where(~(df['regen'] == 'N'), other=False, inplace=True)
df['regen'].where(~(df['regen'] == 'Y'), other=True, inplace=True)
df['regen'] = df['regen'].astype(bool)

# Set the index column to be the day, since it's the unique ID of each row. Also sort sequentially
df.sort_values(by='day', inplace=True)
df.set_index('day', inplace=True)

# Use Reflector of "None" for NaN values and treat the reflector column like a string column
df['hit reflector'] = df['reflector'] >= 1
df['reflector_str'] = df['reflector'].fillna('None')
df['reflector_str'] = df['reflector'].astype(str)
df['reflector_str'].replace(to_replace=['1.0'], value='Reflector #1', inplace=True)
df['reflector_str'].replace(to_replace=['2.0'], value='Reflector #2', inplace=True)
df['reflector_str'].replace(to_replace=['3.0'], value='Reflector #3', inplace=True)
df['reflector_str'].replace(to_replace=['4.0'], value='Reflector #4', inplace=True)
df['reflector_str'].replace(to_replace=['5.0'], value='Reflector #5', inplace=True)

df.head()

,regen,battery,reflector,distance,data,radiation,hit reflector,reflector_str
day,,,,,,,,
1,False,89.4,5.0,1296,241,10.3,True,Reflector #5
2,False,88.6,NaN,1291,254,10.2,False,nan
3,True,91.9,1.0,1334,169,10.2,True,Reflector #1
4,False,89.9,NaN,1301,214,10.3,False,nan
5,True,90.5,4.0,1203,204,10.3,True,Reflector #4


In [406]:
# Engineer a battery delta column
df['battery delta'] = df['battery'].diff()

# Replace the first NaN value with a 0
df['battery delta'] = df['battery delta'].fillna(0)

df.head()

,regen,battery,reflector,distance,data,radiation,hit reflector,reflector_str,battery delta
day,,,,,,,,,
1,False,89.4,5.0,1296,241,10.3,True,Reflector #5,0.0
2,False,88.6,NaN,1291,254,10.2,False,nan,-0.8
3,True,91.9,1.0,1334,169,10.2,True,Reflector #1,3.3
4,False,89.9,NaN,1301,214,10.3,False,nan,-2.0
5,True,90.5,4.0,1203,204,10.3,True,Reflector #4,0.6


In [407]:
df.describe()

,battery,reflector,distance,data,radiation,battery delta
count,365.000000,196.000000,365.000000,365.000000,365.000000,365.000000
mean,85.959178,2.923469,1227.616438,315.205479,10.238904,-0.020548
std,2.706698,1.463841,59.161037,74.287413,0.225000,1.994316
min,78.400000,1.000000,1061.000000,105.000000,9.700000,-5.500000
25%,84.000000,1.750000,1189.000000,262.000000,10.100000,-1.200000
50%,86.000000,3.000000,1228.000000,317.000000,10.200000,0.100000
75%,88.000000,4.000000,1268.000000,367.000000,10.300000,1.100000
max,92.700000,5.000000,1377.000000,497.000000,11.600000,5.900000


In [408]:
import plotly.express as px # Data visualization tool

plotly_options = {
    'labels': {
        'battery': 'Beginning of day Battery %',
        'battery delta': 'Relative Change in Starting Battery %',
        'day': 'Mission Day',
        'regen': 'Regenerated?',
        'distance': 'Distance Travelled (km)',
        'radiation': 'Radiation (Gy)',
        'data': 'Data Transmitted (kb)',
        'reflector': 'Reflector',
        'reflector_str': 'Reflector',
        'hit reflector': 'Hit Reflector?',
    }
}

histogram_options = {
    'color_discrete_sequence':px.colors.sequential.Agsunset,
    'height': 500,
    **plotly_options
}

In [409]:
px.histogram(df, x='battery', title='Start Battery % Distribution', marginal='violin', **histogram_options)

In [410]:
px.histogram(df, x='battery delta', title='Daily Battery Change', marginal='violin', **histogram_options)

In [411]:
px.histogram(df, x='regen', title='Regenerated?', color='regen', **histogram_options)

In [412]:
px.histogram(df, x='hit reflector', title='Relationship of Regeneration and Relfectors', color='regen', facet_col='regen', **histogram_options)

In [413]:
px.histogram(df.sort_values('reflector_str', ascending=True), x='reflector_str', title='Reflector Hit', color='reflector_str', **histogram_options)

In [414]:
px.histogram(df.sort_values('reflector_str'), x='battery delta', title='Battery Recharge by Reflector Hit', color='hit reflector', nbins=75, **histogram_options)

In [415]:
px.histogram(df.sort_values('reflector_str'), x='battery delta', title='Battery Recharge by Reflector Hit', color='reflector_str', nbins=75, **histogram_options)

In [416]:
px.box(df.sort_values('reflector_str'), x='battery delta', y='reflector_str', title='Battery Recharge by Reflector Hit', color='reflector_str', points='suspectedoutliers', **histogram_options)

In [417]:
px.histogram(df, x='battery delta', title='Battery Change by Regenerated', color="regen", facet_col='regen', **histogram_options)

In [418]:
px.histogram(df, x='distance', title='Distance Travelled by Regeneration', color="regen", facet_col='regen', **histogram_options)

In [419]:
px.box(df, y='distance', x='regen', title='Distance Travelled', color='regen', points='outliers', **histogram_options)

In [420]:
px.histogram(df, x='radiation', title='Radiation Encountered', marginal='violin', **histogram_options)

In [421]:
px.histogram(df, x='data', title='Data Transmitted', marginal='violin', color='regen', **histogram_options)

In [422]:
px.line(df, x=df.index, y='distance', title='Daily Distance Travelled', color_discrete_sequence=px.colors.qualitative.Alphabet_r, **plotly_options)

In [423]:
px.line(df, x=df.index, y='battery', title='Daily Starting Battery', color='regen', color_discrete_sequence=px.colors.qualitative.Dark24_r, **plotly_options)

In [424]:
px.line(df, x=df.index, y='data', title='Daily Data Transmitted', color='regen', color_discrete_sequence=px.colors.qualitative.Dark24_r, **plotly_options)

In [425]:
corr = df.corr()

corr

,regen,battery,reflector,distance,data,radiation,hit reflector,battery delta
regen,1.000000,0.260619,-0.101845,0.193348,-0.224850,0.005224,0.994508,0.789572
battery,0.260619,1.000000,-0.239058,0.773549,-0.746683,-0.004332,0.267716,0.369623
reflector,-0.101845,-0.239058,1.000000,-0.166443,0.209700,0.034807,NaN,-0.902655
distance,0.193348,0.773549,-0.166443,1.000000,-0.560387,0.061224,0.199783,0.266635
data,-0.224850,-0.746683,0.209700,-0.560387,1.000000,0.017189,-0.230434,-0.275578
radiation,0.005224,-0.004332,0.034807,0.061224,0.017189,1.000000,0.006720,-0.023805
hit reflector,0.994508,0.267716,NaN,0.199783,-0.230434,0.006720,1.000000,0.789938
battery delta,0.789572,0.369623,-0.902655,0.266635,-0.275578,-0.023805,0.789938,1.000000


In [426]:
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff

z = np.array(corr)

fig = ff.create_annotated_heatmap(
        z,
        x = list(corr.columns),
        y = list(corr.index),
        annotation_text = np.around(z, decimals=2),
        hoverinfo='z',
        colorscale=px.colors.diverging.balance,
    )

fig.update_layout(title='Correlations Analysis Matrix')

fig.show()